In [1]:
import jupyterthemes as jt
from jupyterthemes import jtplot
jtplot.style(theme='onedork')
!jt -t onedork -ofs 12 -dfs 12 -fs 12 -cellw 100% -T # onedork, grade3
#!jt -r
# source activate rdkit && 

In [2]:
import re, os, glob
import image_slicer
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="5,6,7,8"
os.environ["CUDA_VISIBLE_DEVICES"]="8"

In [3]:
from fastai import *
from fastai.vision import *
from fastai.core import *
from fastai.callbacks.hooks import *
from fastai.metrics import error_rate
import matplotlib.pyplot as plt
import re, os, glob
import torch
from PIL import Image

In [4]:
from patches import extract_patches_2d
from patches import reconstruct_from_patches_2d

In [5]:
base_path = os.path.join('/mnt', 'sdf')
path_to_images = os.path.join(base_path, 'training_data')
path_to_dfs = os.path.join(base_path, 'training_data', 'dfs')

In [6]:
# create a single dataframe with all file names and labels
dfs_filenames = glob.glob(os.path.join(path_to_dfs, '*.csv'))  
# return a list of csv in the path
# each csv name represents a candidate drug name?

dfs = []
for filename_i in dfs_filenames:
    dfs.append(pd.read_csv(filename_i))
    
dfs = pd.concat(dfs, axis=0)

In [7]:
# validation of the data set

a_0 = dfs.filename_new.tolist()
a_0_new = ['/mnt/sdf/training_data/' + i for i in a_0]
b_0 = glob.glob(os.path.join(path_to_images, '*.tif'))

# more file names than actual image files

missing_files = list(set(a_0_new) - set(b_0))
missing_files = [re.sub('/mnt/sdf/training_data/', '',i) for i in missing_files]
missing_files += ['caroline_2018 fixations_4 - april_cmap 5 - fix 15-3-18_3 - doxylamine_40x_6-4-18_40x_assay plate 0_3_2018.04.06.20.10.49_d - 4(fld 28 wv red - cy5).tif']

In [8]:
dfs = dfs[~(dfs.filename_new.isin(missing_files))].copy()

In [9]:
dfs.treatment.value_counts()

ab     8264
veh    8260
Name: treatment, dtype: int64

# Splitting images

In [10]:
import torch
from torch.utils import data
import torchvision
import cv2
from torchvision import transforms

In [11]:
def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')

In [12]:
transform1 = transforms.ToTensor()

In [13]:
class SegmentedDataset(data.Dataset):
    def __init__(self, images, labels, segments_per_image):
        self.images = images
        self.labels = labels
        self.segments_per_image = segments_per_image
        self.c = 2
        
    def get_segment(self, image, segment_ID):
        # returns one segment of one image
        a = image.chunk(2, dim=1)
        if segment_ID < 3:
            b = a[0].chunk(2, dim=2)
            return b[segment_ID-1]
        else: 
            b = a[1].chunk(2, dim=2)
            return b[segment_ID-3]   #pil_loader('/mnt/sdf/training_data/'+image)


    def __len__(self):
        'Denotes the total number of samples'
        return len(self.labels)*self.segments_per_image

    def __getitem__(self, index):
        'Generates one sample of data'
        segment_ID = index%self.segments_per_image
        image_ID = index//self.segments_per_image

        image = self.images[image_ID]
        image = pil_loader('/mnt/sdf/training_data/'+image)  #cv2.imread('/mnt/sdf/training_data/'+image)
        image = transform1(image)
        # data and label
        X = self.get_segment(image, segment_ID)
        y = self.labels[image_ID]
        return X, y

In [14]:
training_set = SegmentedDataset(dfs['filename_new'][0:100], dfs['treatment'][0:100],4)
training_generator = data.DataLoader(training_set)# arguments like batch size
len(training_generator)

400

In [15]:
validation_set = SegmentedDataset(dfs['filename_new'][101:200], dfs['treatment'][101:200],4)
validation_generator = data.DataLoader(validation_set) # arguments like batch size

In [16]:
training_set

In [17]:
imdata = ImageDataBunch.create(train_ds=training_set, valid_ds=validation_set, dl_tfms=transform1) #other arguments

In [22]:
for xb,yb in progress_bar(imdata.train_dl):
    print(xb)

RuntimeError: Traceback (most recent call last):
  File "/home/alejo/anaconda3/envs/adam/lib/python3.7/site-packages/torch/utils/data/_utils/pin_memory.py", line 41, in _pin_memory_loop
    batch = pin_memory_batch(batch)
  File "/home/alejo/anaconda3/envs/adam/lib/python3.7/site-packages/torch/utils/data/_utils/pin_memory.py", line 58, in pin_memory_batch
    return [pin_memory_batch(sample) for sample in batch]
  File "/home/alejo/anaconda3/envs/adam/lib/python3.7/site-packages/torch/utils/data/_utils/pin_memory.py", line 58, in <listcomp>
    return [pin_memory_batch(sample) for sample in batch]
  File "/home/alejo/anaconda3/envs/adam/lib/python3.7/site-packages/torch/utils/data/_utils/pin_memory.py", line 50, in pin_memory_batch
    return batch.pin_memory()
RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1556653215914/work/aten/src/THC/THCCachingHostAllocator.cpp:265


In [42]:
learn = cnn_learner(imdata, 
                    models.densenet161, 
                    metrics=accuracy, 
                    bn_final=True,
                    model_dir='/mnt/sdf/adam/models')   # BN at final layer
learn.model = torch.nn.DataParallel(learn.model)

In [43]:
learn.lr_find()
learn.recorder.plot()
torch.cuda.empty_cache()

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


AttributeError: 'str' object has no attribute 'to'